# SunBlock: Data Processing

In [1]:
import sqlite3
import datetime
import pandas as pd 
import plotly.express as px
import plotly.offline as pyo
from statistics import mean
import numpy as np

pyo.init_notebook_mode(connected=True)

## Methods 

In [2]:
## Database Methods 
def DB_init(db_path):
    global DB_CONNECTION, DB_CURSOR
    return sqlite3.connect(db_path)

def DB_connected():
    if DB_CONNECTION != None and DB_CURSOR != None:
        return True
    return False

def DB_insert(data):
    global DB_CURSOR, DB_CONNECTION
    
    if check_db_connection():
        db_query = "INSERT INTO " + DB_TABLE_NAME + " VALUES " + data
        DB_CURSOR.execute(db_query)
        DB_CONNECTION.commit()
    else:
        print("Error: DB Connection not Initialized")


def DB_cleanup():
    if DB_connected():
        DB_CONNECTION.close()

In [3]:
def parsefloat(str_val):
    if isinstance(str_val, float):
        return str_val
    if str_val:
        if str_val[0] == ".":
            return float(str_val[1:])
        if str_val[-1] == ".":
            return float(str_val[0:len(str_val)-1])
        return float(str_val)

In [4]:
def load_solardata(start_date=None, end_date=None):

    # Load data from DB and parse it into a dictionary containing call columns. 
    solar_data = {Timestamp: [], PVVoltage: [], PVCurrent: [], PVPower: [],
               BattVoltage: [], BattChargeCurrent: [], BattChargePower: [],
               LoadPower: [], BattPercentage: [], BattOverallCurrent: [], CPUPowerDraw: [], PowerProfile: []}
    
    max_thresholds = {
        PVVoltage: 25,
        PVCurrent: 5, 
        PVPower: 100, 
        BattVoltage: 15,
        
    }
                  
    if DB_connected():
        query = DB_CURSOR.execute("SELECT * FROM " + DB_TABLE_NAME)
        names = [description[0] for description in DB_CURSOR.description]
            
        for row in query:
            
            #timestamp
            solar_data[Timestamp].append(datetime.datetime.strptime(row[0], DATESTRING_FORMAT))

            #PV Voltage Lowpass filtering
            if parsefloat(row[1]) > max_thresholds[PVVoltage]:
                solar_data[PVVoltage].append(solar_data[PVVoltage][-1])
            else:
                solar_data[PVVoltage].append(parsefloat(row[1]))

            #PVCurrent Lowpass filtering
            if parsefloat(row[2]) > max_thresholds[PVCurrent]:
                solar_data[PVCurrent].append(solar_data[PVCurrent][-1])
            else:
                solar_data[PVCurrent].append(parsefloat(row[2]))

            #PVPower Lowpass Filtering 
            if parsefloat(row[3]) > max_thresholds[PVPower]:
                solar_data[PVPower].append(solar_data[PVPower][-1])
            else:
                solar_data[PVPower].append(parsefloat(row[3]))

            #BattVoltage Lowpass Filtering 
            if parsefloat(row[4]) > max_thresholds[BattVoltage]:
                solar_data[BattVoltage].append(solar_data[BattVoltage][-1])
            else:
                solar_data[BattVoltage].append(parsefloat(row[4]))

            solar_data[BattChargeCurrent].append(row[5])
            solar_data[BattChargePower].append(row[6])
            solar_data[LoadPower].append(row[7])
            solar_data[BattPercentage].append(row[8])
            solar_data[BattOverallCurrent].append(row[9])
            solar_data[CPUPowerDraw].append(row[10])
            solar_data[PowerProfile].append(row[11])
    else:
        print("Error: DB Connection not Initialized")


    DB_cleanup()
    return pd.DataFrame(solar_data)


In [5]:
# Window functions 
def window(start_date, end_date=None):
    # iput: start and end dates to be Timestamp objects. 
    # output: returns tuple with indices to slice Dataframe at
    
    slice_start_index = -1
    slice_end_index = len(DATAFRAME[Timestamp]) 
    
    for i in range(len(DATAFRAME[Timestamp])):
        
        if slice_start_index < 0 and DATAFRAME[Timestamp][i].date() >= start_date:
            slice_start_index = i
            if not end_date:
                return (slice_start_index, len(DATAFRAME[Timestamp])) 
        
        if slice_start_index > -1 and DATAFRAME[Timestamp][i].date() > end_date:
            slice_end_index = i - 1 
            return (slice_start_index, slice_end_index)
    return (slice_start_index, slice_end_index)


def resample(df, step):
    # iput:  takes a dataframe and sampling frequency
    # oput:  a new dataframe 
    return df.iloc[0: len(df):step]

In [6]:
## Calculations
def coulomb_count():
    return sum(resampled_df[PVPower]) * SAMPLING_FREQUENCY / mean(resampled_df[PVVoltage])

def peak_value(val):
    return max(resampled_df[val])

## Constants

In [7]:
DB_CONNECTION = None
DB_CURSOR = None 

DATESTRING_FORMAT = '%Y-%m-%d %H:%M:%S'
DATAFRAME = None


## Columns 
Timestamp = "Timestamp"

PVVoltage = "PVVoltage"
PVCurrent = "PVCurrent"
PVPower = "PVPower"

BattVoltage = "BattVoltage"
BattChargeCurrent = "BattChargeCurrent"
BattChargePower = "BattChargePower"
BattPercentage = "BattPercentage"
BattOverallCurrent = "BattOverallCurrent"

LoadPower = "LoadPower"
CPUPowerDraw = "CPUPowerDraw"

PowerProfile = "PowerProfile"

## Loading data from Database

1. Connect to the database
2. Load data into a dataframe
3. Disconnect the database.

From this point onwards, the database is no longer needed. 

In [8]:
DB_PATH = "/Users/muhammadshahromali/SunblockData/sb1.db"
DB_TABLE_NAME = "solardata"
DB_CONNECTION = DB_init(DB_PATH)
DB_CURSOR = DB_CONNECTION.cursor()


DATAFRAME = load_solardata()


In [9]:
# 4 - 8 

start_date = datetime.date(2025, 5, 5)
end_date = datetime.date(2025, 5, 11)
df_slice = window(start_date, end_date)

In [10]:
SAMPLING_FREQUENCY = 60  # in seconds
resampled_df = resample(DATAFRAME.loc[df_slice[0]:df_slice[1]], SAMPLING_FREQUENCY)

## Plotting with Plotly
Using plotly.express
[Plotly docs](https://plotly.com/python/line-charts/)

In [11]:
fig = px.line(resampled_df, x=Timestamp, y=[BattPercentage, LoadPower, PVPower])
fig.show()

# fig = px.line(resampled_df, x=Timestamp, y=[PVPower])
# fig.update_traces(line_color='#008F00')
# fig.show()

# fig = px.line(resampled_df, x=Timestamp, y=[BattChargeCurrent, PVCurrent, PVVoltage, PVPower])
# # fig.update_traces(line_color='#0000FF')
# fig.show()

# fig = px.line(resampled_df, x=Timestamp, y=[LoadPower])
# fig.update_traces(line_color='#ff0000')
# fig.show()



In [13]:
# print("Maximum Power Generated for period was", max(resampled_df[PVPower]), "watts") 
# print("Average Power Generated per minute", mean(resampled_df[PVPower]), "watts")
# print("(Unconfirmed Val) Charge Generated for period:") 
# print("Average Power draw for period was", mean(resampled_df[LoadPower]), "watts")